# 1. Basico: Carga de Datos

Importamos las librerias

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
data = pd.read_csv('/content/OnlineRetail.csv', sep=';')
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France




*  Al ejecutar tu analisis exploratorio, describe el numero de columnas y filas que encuentras, en cuales encuentras valores nulos o perdidos y cuantos, Para los valores perdidos numericos:

Numero de filas y columnas

In [3]:
print('El numero de filas es: ', data.shape[0])
print('El numero de columnas es: ', data.shape[1])

El numero de filas es:  541909
El numero de columnas es:  8


Valores nulos o perdidos

In [4]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64



*   **Elimina todos los valores perdidos utilizando .dropna**

In [8]:
data.dropna(inplace=True)
print('se elimino correctamente')

se elimino correctamente




*   **Calcula la media, maximo, minimo y desviacion estandar de las columnas con valores numericos.**

In [6]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000




*   **Si encuentras un valor que no tenga logica, es decir valores negativos en columnas donde no deberian haber, eliminalos.**

In [9]:
print('La cantidad minima nunca debe ser menor a cero')
data = data[data.Quantity >= 0]
data.describe()


La cantidad minima nunca debe ser menor a cero


,Quantity,UnitPrice,CustomerID
count,397924.000000,397924.000000,397924.000000
mean,13.021823,3.116174,15294.315171
std,180.420210,22.096788,1713.169877
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


*   **Calcula cuantos valores unicos encontraste dentro de cada columna despues de limpiar los datos.

In [10]:
data.nunique()

InvoiceNo      18536
StockCode       3665
Description     3877
Quantity         302
InvoiceDate    17286
UnitPrice        441
CustomerID      4339
Country           37
dtype: int64

# 2. Intermedio: RFM

*   **Calcula la Recenty (Reciente), Frecuency (Frecuencia) y Monetary value (Valor monetario)**

In [20]:
#Aplicando el tipo de dato a la columna para que se pueda trabajar con las fechas
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Recenty

In [21]:
df_recency = data.groupby(by='CustomerID', as_index=False)['InvoiceDate'].max() 
df_recency.columns = ['CustomerID', 'LastPurchaseDate']
recent_date = df_recency['LastPurchaseDate'].max()
df_recency['Recency'] = df_recency['LastPurchaseDate'].apply(lambda x: (recent_date - x).days)
df_recency.head() #Mostramos resultados.

,CustomerID,LastPurchaseDate,Recency
0,12346.0,2011-01-18 10:01:00,325
1,12347.0,2011-12-07 15:52:00,1
2,12348.0,2011-09-25 13:13:00,74
3,12349.0,2011-11-21 09:51:00,18
4,12350.0,2011-02-02 16:01:00,309


Frecuency

In [22]:
frequency_df = data.drop_duplicates().groupby(by=['CustomerID'], as_index=False)['InvoiceDate'].count() #Agrupamos por CustomerID sin duplicados
frequency_df.columns = ['CustomerID', 'Frequency'] #Nombramos columnas.
frequency_df.head(10)

,CustomerID,Frequency
0,12346.0,1
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17
5,12352.0,85
6,12353.0,4
7,12354.0,58
8,12355.0,13
9,12356.0,59


Monetary Value

In [23]:
data['Total'] = data['UnitPrice'] * data['Quantity'] # Calculamos totales multiplicando precio unitario * cantidad
monetary_df = data.groupby(by = 'CustomerID', as_index=False)['Total'].sum() #Agrupamos por CustomerID
monetary_df.columns = ['CustomerID', 'Monetary'] #Creamos columnas
monetary_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CustomerID,Monetary
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40


*   **Incluye estos valores en una tabla**

In [24]:
print('Tabla de Valores')
dataframe = df_recency.merge(frequency_df, on = 'CustomerID')
df_rfm = dataframe.merge(monetary_df, on='CustomerID').drop(columns='LastPurchaseDate')
df_rfm.head()

Tabla de Valores


,CustomerID,Recency,Frequency,Monetary
0,12346.0,325,1,77183.60
1,12347.0,1,182,4310.00
2,12348.0,74,31,1797.24
3,12349.0,18,73,1757.55
4,12350.0,309,17,334.40


*  **Calcula los puntajes o scores con un rango de 1 a 5.**

In [25]:
df_rfm['r_quartile'] = pd.qcut(df_rfm['Recency'], 5, ['5','4','3','2','1'])
df_rfm['f_quartile'] = pd.qcut(df_rfm['Frequency'], 5, ['1','2','3','4','5'])
df_rfm['m_quartile'] = pd.qcut(df_rfm['Monetary'], 5, ['1','2','3','4','5'])
display(df_rfm)

,CustomerID,Recency,Frequency,Monetary,r_quartile,f_quartile,m_quartile
0,12346.0,325,1,77183.60,1,1,5
1,12347.0,1,182,4310.00,5,5,5
2,12348.0,74,31,1797.24,2,3,4
3,12349.0,18,73,1757.55,4,4,4
4,12350.0,309,17,334.40,1,2,2
...,...,...,...,...,...,...,...
4334,18280.0,277,10,180.60,1,1,1
4335,18281.0,180,7,80.82,1,1,1
4336,18282.0,7,12,178.05,5,1,1
4337,18283.0,3,721,2094.88,5,5,5
